In [10]:
import mxnet as mx
import numpy as np
import sys
import networkx as nx

In [6]:
class AutoEncoderModel:
    def __init__(self, dims, internal_act=None):
        self.data = mx.symbol.Variable('data')
        self.y = mx.symbol.Variable('label')
        self.fc1_weight = mx.symbol.Variable('fc1_weight')
        self.fc1_bias = mx.symbol.Variable('fc1_bias')
        self.fc2_weight = mx.symbol.Variable('fc2_weight')
        self.fc2_bias = mx.symbol.Variable('fc2_bias')
        x = mx.symbol.FullyConnected(data=self.data, weight=self.fc1_weight,
                                     bias=self.fc1_bias, num_hidden=dims[1])
        self.layer1 = x
        if (internal_act is not None):
            x = mx.symbol.Activation(data=x, act_type=internal_act)
        x = mx.symbol.FullyConnected(data=x, weight=self.fc2_weight,
                                     bias=self.fc2_bias, num_hidden=dims[2])
        self.layer2 = x
        self.loss = mx.symbol.LinearRegressionOutput(data=x, label=self.y)

    def fit(self, data, batch_size, params=None):
        data_iter = mx.io.NDArrayIter(data={'data':data}, label={'label':data},
                batch_size=batch_size, shuffle=True,
                last_batch_handle='roll_over')
        self.model = mx.mod.Module(symbol=self.loss, data_names=['data'], label_names = ['label'])
        self.model.bind(data_shapes=data_iter.provide_data, label_shapes=data_iter.provide_label)
        if (params is not None):
            self.model.set_params(arg_params=params, aux_params=None, force_init=True)
        self.model.fit(data_iter, optimizer_params={'learning_rate':0.005, 'momentum': 0.9},
                num_epoch=50, eval_metric='mse',
                batch_end_callback = mx.callback.Speedometer(batch_size, 2))

In [7]:
def train(data, num_epoc, internal_act=None):
    int_epoc = 100
    params = None
    for i in range(num_epoc/int_epoc):
        model = AutoEncoderModel([data.shape[1], 10, data.shape[1]])
        model.fit(data, int_epoc, params)

        params = model.model.get_params()[0]
        fc1_weight = params.get('fc1_weight').asnumpy()
        fc1_bias = params.get('fc1_bias').asnumpy()
        fc2_weight = params.get('fc2_weight').asnumpy()
        fc2_bias = params.get('fc2_bias').asnumpy()

        np_data = data.asnumpy()
        hidden = np.dot(np_data, fc1_weight.T) + fc1_bias
        output = np.dot(hidden, fc2_weight.T) + fc2_bias
        print("epoc " + str(i * int_epoc) + ": " + str(np.sum(np.square(output - np_data))))
        sys.stdout.flush()
    return params

In [8]:
data1 = mx.ndarray.random_uniform(shape=[10000, 10])
data2 = mx.ndarray.random_uniform(shape=[10, 100])
data = mx.ndarray.dot(data1, data2)
print(data.shape)

(10000L, 100L)


## Run on real data

We compute the embedding on a graph with 81306 vertices and 1768149 vertices. To embed the graph into 10 dimensions, we start with the most densest columns and increase the number of columns to embed. When we increase the number of columns to embed, we use the parameters trained from the previous run (on the dataset with a smaller number of columns).

In [44]:
elg = nx.read_edgelist("/home/ubuntu/datasets/twitter_combined.txt")
spm = nx.to_scipy_sparse_matrix(elg)
colsum = np.ravel(spm.sum(axis=1))

### Compute the embedding on the densest 10 columns.

In [51]:
max10 = np.sort(np.ravel(colsum), axis=None)[len(colsum) - 10]
data10 = mx.ndarray.sparse.csr_matrix(spm[:,colsum >= max10])
print(mx.ndarray.sum(data, axis=0))
print(data.shape)


[ 2490.  3383.  2484.  2758.  2476.  1789.  2133.  3011.  3239.  2155.]
<NDArray 10 @cpu(0)>
(81306L, 10L)


In [60]:
params_linear10=train(data10, 5000, internal_act=None)

epoc 0: 3093.22


epoc 100: 1652.24


epoc 200: 814.882


epoc 300: 511.462


epoc 400: 343.608


...

epoc 4500: 0.00100734


epoc 4600: 0.000975417


epoc 4700: 0.00092864


epoc 4800: 0.000900516


epoc 4900: 0.000881277


In [52]:
params_sigmoid10=train(data10, 5000, internal_act='sigmod')

epoc 0: 3203.39


epoc 100: 955.346


epoc 200: 834.44


epoc 300: 581.168


epoc 400: 335.639


...

epoc 4500: 0.000948211


epoc 4600: 0.000939742


epoc 4700: 0.000920065


epoc 4800: 0.000907595


epoc 4900: 0.000887442


In [53]:
params_relu10=train(data10, 5000, internal_act='relu')

epoc 0: 3573.51


epoc 100: 956.345


epoc 200: 846.814


epoc 300: 635.075


epoc 400: 348.802


...

epoc 4500: 0.00116306


epoc 4600: 0.00111296


epoc 4700: 0.00108219


epoc 4800: 0.00105612


epoc 4900: 0.0010254


### Compute the embedding on the densest 30 columns.

In [55]:
max30 = np.sort(np.ravel(colsum), axis=None)[len(colsum) - 30]
data30 = mx.ndarray.sparse.csr_matrix(spm[:,colsum >= max30])
print(mx.ndarray.sum(data30, axis=0))
print(data30.shape)


[ 1256.  1377.  1467.  1251.  1521.  1229.  2490.  1695.  1291.  1387.
  1275.  1743.  1443.  1497.  3383.  2484.  2758.  1666.  1255.  2476.
  1789.  1358.  1269.  1395.  2133.  3011.  3239.  1568.  2155.  1509.]
<NDArray 30 @cpu(0)>
(81306L, 30L)


In [57]:
params_linear30=train(data30, 5000, internal_act=None)

epoc 0: 36343.5


epoc 100: 28741.8


epoc 200: 24386.4


epoc 300: 20273.6


epoc 400: 19463.6


...

epoc 4500: 19081.2


epoc 4600: 19081.2


epoc 4700: 19081.1


epoc 4800: 19081.1


epoc 4900: 19081.1


In [58]:
params_linear30=train(data30, 5000, internal_act='sigmod')

epoc 0: 35963.1


epoc 100: 28377.2


epoc 200: 23608.4


epoc 300: 20077.8


epoc 400: 19401.5


...

epoc 4500: 19081.4


epoc 4600: 19081.3


epoc 4700: 19081.3


epoc 4800: 19081.2


epoc 4900: 19081.1
